### csv 한줄씩 gpt가 읽고 labeling 하기 
비용 최적화 -> 단어개수 30개로 제한, 한번에 20행 묶어서 질의하여 토큰을 아낀다.

제 짠내나는 코드가 도움이 되면 좋겠습니다.. ^^ 

In [2]:
%pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [15]:
# 필요한 라이브러리 설치 및 가져오기
import os
import pandas as pd
import openai
from time import sleep
from IPython.display import clear_output  # 출력 내용을 지우기 위해 사용

# OpenAI API 키 설정
openai.api_key = "{여기 지피티 api key를 작성하세요!}"

# CSV 파일 읽기 
targetNum = 35
file_path = f'final_data/final_data_split_{targetNum}.csv'  # CSV 파일 경로를 여기에 입력하세요
output_path = f'labeling_{targetNum}_gpt.csv'

In [16]:
# 데이터 로드
data = pd.read_csv(file_path)

# 새로운 'label' 열 추가, 기본값은 0
data['label'] = 0 # 기본값 0

# 초기 저장 (헤더 포함)
data.to_csv(output_path, index=False, encoding='utf-8-sig')

# 텍스트를 최대 30단어로 제한하는 함수
def limit_to_30_words(content):
    """
    텍스트에서 최대 30단어까지만 추출합니다.
    텍스트가 비어있거나 단어가 30개 미만이면 그대로 사용함
    """
    if not isinstance(content, str) or not content.strip():
        return ""  # 텍스트가 비어 있으면 빈 문자열 반환

    words = content.split()  # 공백 기준으로 단어 분리
    limited_content = ' '.join(words[:30]).strip()  # 최대 30단어까지 유지
    return limited_content

# GPT 판단 함수
def gpt_labeling_batch(contents):
    # 번호와 텍스트를 묶어서 프롬프트 생성
    prompt = "You are analyzing multiple pieces of content to determine if they were written by a sex offender (rapist).\n"
    prompt += "- Respond `1` if it is written by a sex offender.\n"
    prompt += "- Respond `0` if it is not clear or written by someone else.\n\n"
    prompt += "Contents:\n"
    prompt += "\n".join([f"{i + 1}. {content}" for i, content in enumerate(contents)])
    prompt += "\n\nPlease respond with a sequence of `0` and `1` representing each content (e.g., `01001`)."

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", # 모델도 성능과 비용을 보고 바꾸시면 됩니다. gpt-4o는 비싸지만 성능이 더 좋습니다. 
            messages=[
                {"role": "system", "content": "You are an assistant for content analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # 낮은 온도로 일관성 있는 결과를 유지
        )

        gpt_response = response['choices'][0]['message']['content'].strip()
        
        # 응답을 숫자 리스트로 변환
        labels = [int(char) for char in gpt_response if char in '01']
        return labels
    
    except Exception as e:
        print(f"예상치 못한 오류: {e}")
        return -1

In [17]:
# 데이터 처리 및 저장
batch_size = 20  # 한 요청당 처리할 행 개수

for start in range(0, len(data), batch_size):
    # 현재 배치 데이터 추출
    end = min(start + batch_size, len(data))
    batch_contents = [limit_to_30_words(data.loc[i, 'content']) for i in range(start, end)]
    
    # GPT 라벨링 요청
    batch_labels = gpt_labeling_batch(batch_contents)
    
    # 결과 길이 확인 및 조정
    if len(batch_labels) != (end - start):
        print(f"Warning: Mismatch in lengths. Adjusting batch_labels...")
        if len(batch_labels) < (end - start):  # 부족하면 패딩 추가
            batch_labels += [-1] * ((end - start) - len(batch_labels))
        else:  # 초과하면 자르기
            batch_labels = batch_labels[:(end - start)]
    
    # 결과 저장
    data.loc[start:end-1, 'label'] = batch_labels

    # 중간 저장
    data.iloc[:end].to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"{end}개 행 저장 완료")

print(f"라벨링이 완료되었습니다. 결과는 '{output_path}'에 저장되었습니다.")


20개 행 저장 완료
40개 행 저장 완료
60개 행 저장 완료
80개 행 저장 완료
100개 행 저장 완료
120개 행 저장 완료
140개 행 저장 완료
160개 행 저장 완료
180개 행 저장 완료
200개 행 저장 완료
220개 행 저장 완료
240개 행 저장 완료
260개 행 저장 완료
280개 행 저장 완료
300개 행 저장 완료
320개 행 저장 완료
340개 행 저장 완료
360개 행 저장 완료
380개 행 저장 완료
400개 행 저장 완료
420개 행 저장 완료
440개 행 저장 완료
460개 행 저장 완료
480개 행 저장 완료
500개 행 저장 완료
520개 행 저장 완료
540개 행 저장 완료
556개 행 저장 완료
라벨링이 완료되었습니다. 결과는 'labeling_34_gpt.csv'에 저장되었습니다.


### 아래는 통합코드
위는 가독성을 위해 부분 별로 셀을 나누어 둔 것입니다.
편의성은 아래 통합된 코드가 나을 것으로 생각됩니다. 

In [34]:
# CSV 파일 읽기 
targetNum = 50
file_path = f'final_data/final_data_split_{targetNum}.csv'  # CSV 파일 경로를 여기에 입력하세요
output_path = f'labeling_{targetNum}_gpt.csv'

# 데이터 로드
data = pd.read_csv(file_path)

# 새로운 'label' 열 추가, 기본값은 0
data['label'] = 0 # 기본값 0

# 초기 저장 (헤더 포함)
data.to_csv(output_path, index=False, encoding='utf-8-sig')

# 텍스트를 최대 30단어로 제한하는 함수
def limit_to_30_words(content):
    """
    텍스트에서 최대 30단어까지만 추출합니다.
    텍스트가 비어있거나 단어가 30개 미만이면 그대로 사용함
    """
    if not isinstance(content, str) or not content.strip():
        return ""  # 텍스트가 비어 있으면 빈 문자열 반환

    words = content.split()  # 공백 기준으로 단어 분리
    limited_content = ' '.join(words[:30]).strip()  # 최대 30단어까지 유지
    return limited_content

# GPT 판단 함수
def gpt_labeling_batch(contents):
    # 번호와 텍스트를 묶어서 프롬프트 생성
    prompt = "You are analyzing multiple pieces of content to determine if they were written by a sex offender (rapist).\n"
    prompt += "- Respond `1` if it is written by a sex offender.\n"
    prompt += "- Respond `0` if it is not clear or written by someone else.\n\n"
    prompt += "Contents:\n"
    prompt += "\n".join([f"{i + 1}. {content}" for i, content in enumerate(contents)])
    prompt += "\n\nPlease respond with a sequence of `0` and `1` representing each content (e.g., `01001`)."

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant for content analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8  # 낮은 온도로 일관성 있는 결과를 유지
        )

        gpt_response = response['choices'][0]['message']['content'].strip()
        
        # 응답을 숫자 리스트로 변환
        labels = [int(char) for char in gpt_response if char in '01']
        return labels
    
    except Exception as e:
        print(f"예상치 못한 오류: {e}")
        return -1

# 데이터 처리 및 저장
batch_size = 20  # 한 요청당 처리할 행 개수

for start in range(0, len(data), batch_size):
    # 현재 배치 데이터 추출
    end = min(start + batch_size, len(data))
    batch_contents = [limit_to_30_words(data.loc[i, 'content']) for i in range(start, end)]
    
    # GPT 라벨링 요청
    batch_labels = gpt_labeling_batch(batch_contents)
    
    # 결과 길이 확인 및 조정
    if len(batch_labels) != (end - start):
        print(f"Warning: Mismatch in lengths. Adjusting batch_labels...")
        if len(batch_labels) < (end - start):  # 부족하면 패딩 추가
            batch_labels += [-1] * ((end - start) - len(batch_labels))
        else:  # 초과하면 자르기
            batch_labels = batch_labels[:(end - start)]
    
    # 결과 저장
    data.loc[start:end-1, 'label'] = batch_labels

    # 중간 저장
    data.iloc[:end].to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"{end}개 행 저장 완료")

print(f"라벨링이 완료되었습니다. 결과는 '{output_path}'에 저장되었습니다.")


20개 행 저장 완료
40개 행 저장 완료
60개 행 저장 완료
80개 행 저장 완료
100개 행 저장 완료
120개 행 저장 완료
140개 행 저장 완료
160개 행 저장 완료
180개 행 저장 완료
200개 행 저장 완료
220개 행 저장 완료
240개 행 저장 완료
260개 행 저장 완료
280개 행 저장 완료
300개 행 저장 완료
320개 행 저장 완료
340개 행 저장 완료
360개 행 저장 완료
380개 행 저장 완료
400개 행 저장 완료
420개 행 저장 완료
440개 행 저장 완료
460개 행 저장 완료
480개 행 저장 완료
500개 행 저장 완료
520개 행 저장 완료
540개 행 저장 완료
560개 행 저장 완료
580개 행 저장 완료
583개 행 저장 완료
라벨링이 완료되었습니다. 결과는 'labeling_50_gpt.csv'에 저장되었습니다.
